In [1]:

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50
from model import HFFR_ProtoNet
import torch,cv2
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
model = HFFR_ProtoNet(num_class=200)
model.load_state_dict(torch.load('/home/liangxiaoyuan/小样本学习/logs/pretrain_CUB/best_pretrain.pt'))
# model = resnet50()
model.eval()

model = model.cuda()


/home/liangxiaoyuan/miniconda3/envs/ErrorLog/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_2539748/493343552.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will

In [12]:

def reshape_transform(out):
    # out=out[0].reshape(1,-1)
    # out=out[1]
    print(out.shape)
    # C,W,H=out.shape
    # out=out.reshape(1,C,H,W)
    # print(out.shape)
    return out
# 选择目标层
target_layers = [model.feature_extractor.features.layer4]#[model.local_attention.attention_conv[-1]] [model.feature_extractor.features.layer4] 
# 假设model是你训练好的模型，target_layers是你想可视化的层
cam = GradCAM(model=model, target_layers=target_layers,reshape_transform=reshape_transform)
path='/home/liangxiaoyuan/小样本学习/CUB_200_2011/images/019.Gray_Catbird/Gray_Catbird_0002_21395.jpg'
# 获取图像tensor并标准化处理
rgb_img = Image.open(path).convert('RGB')
input_tensor =transform(rgb_img)[None].cuda()
# 类别索引，可以直接创建目标
targets =[ClassifierOutputTarget(70)]
# 计算cam
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

rgb_img = cv2.imread(path)
rgb_img=cv2.resize(rgb_img,(256,256))/255.0
visualization = show_cam_on_image(rgb_img, grayscale_cam[0], use_rgb=False)
# cv2.imwrite('/home/liangxiaoyuan/小样本学习/注意力可视化/ex3/layer1.jpg',visualization)
cv2.imwrite('/home/liangxiaoyuan/小样本学习/注意力可视化/local_attention.jpg',visualization)

torch.Size([1, 2048, 8, 8])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([1, 2048, 8, 8])


True

In [9]:
model.AdaptiveChoose

FeatureSelect(
  (pro_layer): Linear(in_features=16384, out_features=8192, bias=True)
  (atten): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=8192, out_features=8192, bias=True)
    )
    (linear1): Linear(in_features=8192, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=8192, bias=True)
    (norm1): LayerNorm((8192,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((8192,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (downsampe): Sequential(
    (0): Linear(in_features=8192, out_features=4096, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4096, out_features=2048, bias=True)
    (3): ReLU()
  )
)

In [24]:
path='/home/liangxiaoyuan/小样本学习/注意力可视化/ex3/image.jpg'
img=cv2.resize(cv2.imread(path),(256,256))
cv2.imwrite(path,img)

True